In [ ]:
import os
import subprocess
import sys # Importar sys para forzar el flush de la salida

YOLOX_PATH = r"C:\Users\aarnaizl\Documents\YOLOX"
CONFIG_NAME = "yolox_m"

def train_yolox(config_name):
    exp_file = os.path.join(YOLOX_PATH, "exps", "default", f"{config_name}.py")

    cmd = [
        "python", os.path.join(YOLOX_PATH, "tools", "train.py"),
        "-f", exp_file,
        "-d", "1",
        "-b", "8",
        "--fp16",
        "-o",
        "--cache"
    ]

    print("[INFO] Launching YOLOX training with:")
    print(" ".join(cmd))
    print("-" * 50) # Separador para visualizar mejor el inicio de los logs

    # --- CAMBIO CLAVE AQUÍ: Usar subprocess.Popen para streaming en tiempo real ---
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    # stdout=subprocess.PIPE: Captura la salida estándar
    # stderr=subprocess.stdout: Redirige errores a la misma salida
    # text=True: Decodifica la salida como texto
    # bufsize=1: Configura el buffer de línea (importante para tiempo real)

    # Lee y imprime la salida línea por línea
    while True:
        line = process.stdout.readline()
        if not line:
            break
        print(line, end='') # 'end=''' para evitar dobles saltos de línea
        sys.stdout.flush() # Fuerza la impresión inmediata en Jupyter

    # Espera a que el proceso termine y obtiene el código de retorno
    process.wait()
    print("-" * 50)
    print(f"[INFO] Training process exited with code: {process.returncode}")
    # --- FIN CAMBIO ---

train_yolox(CONFIG_NAME)

In [ ]:
import torch; print(torch.__version__); print(torch.version.cuda); print(torch.cuda.is_available())

In [ ]:
import json
import os
from datetime import datetime

def fix_coco_annotation_file(file_path):
    """
    Fix COCO annotation file by adding missing 'info' field if it doesn't exist.
    """
    print(f"Processing: {file_path}")
    
    # Read the current annotation file
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Check if 'info' field exists
    if 'info' not in data:
        print("  - Adding missing 'info' field")
        data['info'] = {
            "description": "Traffic Signs Dataset",
            "url": "",
            "version": "1.0",
            "year": 2025,
            "contributor": "Traffic Signs Detection Project",
            "date_created": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    else:
        print("  - 'info' field already exists")
    
    # Check if 'licenses' field exists (optional but recommended)
    if 'licenses' not in data:
        print("  - Adding missing 'licenses' field")
        data['licenses'] = [
            {
                "url": "",
                "id": 1,
                "name": "Unknown License"
            }
        ]
    else:
        print("  - 'licenses' field already exists")
    
    # Ensure other required fields exist
    required_fields = ['images', 'annotations', 'categories']
    for field in required_fields:
        if field not in data:
            print(f"  - WARNING: Missing required field '{field}'")
            data[field] = []
        else:
            print(f"  - '{field}' field exists with {len(data[field])} items")
    
    # Create backup of original file
    backup_path = file_path + '.backup'
    if not os.path.exists(backup_path):
        print(f"  - Creating backup: {backup_path}")
        with open(backup_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)
    
    # Write the fixed file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)
    
    print(f"  - Fixed file saved: {file_path}")
    return True

def main():
    # Path to your dataset directory
    dataset_dir = "D:/Ainhoa/traffic_signs_data/DFG_detection/dataset_coco_ready_original_yolox/annotations"
    
    # Annotation files to fix
    annotation_files = [
        "coco_annotations.json",
        "coco_val_annotations.json"
    ]
    
    print("=== COCO Annotation Files Fixer ===")
    print(f"Dataset directory: {dataset_dir}")
    print()
    
    for ann_file in annotation_files:
        file_path = os.path.join(dataset_dir, ann_file)
        
        if os.path.exists(file_path):
            try:
                fix_coco_annotation_file(file_path)
                print("  ✓ Successfully fixed!")
            except Exception as e:
                print(f"  ✗ Error fixing {ann_file}: {e}")
        else:
            print(f"  ✗ File not found: {file_path}")
        print()
    
    print("=== Verification ===")
    print("Now let's verify the fixed files:")
    print()
    
    for ann_file in annotation_files:
        file_path = os.path.join(dataset_dir, ann_file)
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                print(f"File: {ann_file}")
                print(f"  - Has 'info' field: {'info' in data}")
                print(f"  - Has 'licenses' field: {'licenses' in data}")
                print(f"  - Has 'images' field: {'images' in data} ({len(data.get('images', []))} items)")
                print(f"  - Has 'annotations' field: {'annotations' in data} ({len(data.get('annotations', []))} items)")
                print(f"  - Has 'categories' field: {'categories' in data} ({len(data.get('categories', []))} items)")
                
                if 'info' in data:
                    print(f"  - Info: {data['info']}")
                print()
            except Exception as e:
                print(f"  ✗ Error reading {ann_file}: {e}")
                print()

if __name__ == "__main__":
    main()

In [ ]:
import os
import subprocess
import sys
import json

YOLOX_PATH = r"C:\Users\aarnaizl\Documents\YOLOX"
CONFIG_NAME = "yolox_m"

def verify_coco_annotations(data_dir, train_ann, val_ann):
    """
    Verify that COCO annotation files have the required structure.
    """
    print("[INFO] Verifying COCO annotation files...")
    
    required_fields = ['info', 'images', 'annotations', 'categories']
    
    for ann_file in [train_ann, val_ann]:
        ann_path = os.path.join(data_dir, ann_file)
        
        if not os.path.exists(ann_path):
            print(f"[ERROR] Annotation file not found: {ann_path}")
            return False
        
        try:
            with open(ann_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print(f"[INFO] Checking {ann_file}:")
            
            for field in required_fields:
                if field not in data:
                    print(f"[ERROR] Missing required field '{field}' in {ann_file}")
                    print(f"[FIX] Please run the COCO annotation fixer first!")
                    return False
                else:
                    count = len(data[field]) if isinstance(data[field], list) else "N/A"
                    print(f"  ✓ {field}: {count} items" if isinstance(count, int) else f"  ✓ {field}: exists")
            
            # Check for empty annotations
            if len(data.get('annotations', [])) == 0:
                print(f"[WARNING] No annotations found in {ann_file}")
            
            if len(data.get('categories', [])) == 0:
                print(f"[ERROR] No categories found in {ann_file}")
                return False
            
            print(f"  ✓ {ann_file} looks good!")
            
        except json.JSONDecodeError as e:
            print(f"[ERROR] Invalid JSON in {ann_file}: {e}")
            return False
        except Exception as e:
            print(f"[ERROR] Error reading {ann_file}: {e}")
            return False
    
    print("[INFO] All annotation files verified successfully!")
    return True

def check_dataset_structure(data_dir):
    """
    Check if the dataset directory structure is correct.
    """
    print(f"[INFO] Checking dataset structure in: {data_dir}")
    
    if not os.path.exists(data_dir):
        print(f"[ERROR] Dataset directory not found: {data_dir}")
        return False
    
    # Check for typical image directories
    image_dirs = ['train2017', 'val2017', 'images', 'train', 'val']
    found_image_dir = False
    
    for img_dir in image_dirs:
        full_path = os.path.join(data_dir, img_dir)
        if os.path.exists(full_path):
            image_count = len([f for f in os.listdir(full_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"  ✓ Found image directory: {img_dir} ({image_count} images)")
            found_image_dir = True
    
    if not found_image_dir:
        print("[WARNING] No standard image directories found. Make sure your images are accessible.")
    
    return True

def train_yolox(config_name):
    """
    Train YOLOX with improved error handling and verification.
    """
    exp_file = os.path.join(YOLOX_PATH, "exps", "default", f"{config_name}.py")
    
    if not os.path.exists(exp_file):
        print(f"[ERROR] Config file not found: {exp_file}")
        return False
    
    # Read the config to get dataset info
    print("[INFO] Reading configuration...")
    try:
        # This is a simple way to extract info from the config
        # In a real scenario, you might want to import it properly
        with open(exp_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extract data_dir, train_ann, val_ann from the config
        import re
        data_dir_match = re.search(r'self\.data_dir\s*=\s*["\']([^"\']+)["\']', content)
        train_ann_match = re.search(r'self\.train_ann\s*=\s*["\']([^"\']+)["\']', content)
        val_ann_match = re.search(r'self\.val_ann\s*=\s*["\']([^"\']+)["\']', content)
        
        if data_dir_match and train_ann_match and val_ann_match:
            data_dir = data_dir_match.group(1)
            train_ann = "annotations/" + train_ann_match.group(1)
            val_ann = "annotations/" + val_ann_match.group(1)
            
            print(f"[INFO] Dataset directory: {data_dir}")
            print(f"[INFO] Train annotations: {train_ann}")
            print(f"[INFO] Val annotations: {val_ann}")
            
            # Verify dataset structure
            if not check_dataset_structure(data_dir):
                return False
            
            # Verify COCO annotations
            if not verify_coco_annotations(data_dir, train_ann, val_ann):
                print("[ERROR] Please fix the COCO annotation files before training!")
                print("[HINT] Run the COCO annotation fixer script first.")
                return False
        else:
            print("[WARNING] Could not extract dataset info from config. Proceeding anyway...")
    
    except Exception as e:
        print(f"[WARNING] Error reading config file: {e}. Proceeding anyway...")
    
    # Prepare training command
    cmd = [
        "python", os.path.join(YOLOX_PATH, "tools", "train.py"),
        "-f", exp_file,
        "-d", "1",           # Use 1 GPU
        "-b", "8",           # Batch size
        "--fp16",            # Use mixed precision
        "-o",                # Use multiscale training
        "--cache"            # Cache images for faster training
    ]
    
    print("[INFO] Launching YOLOX training with:")
    print(" ".join(cmd))
    print("-" * 50)
    
    try:
        # Use subprocess.Popen for real-time output streaming
        process = subprocess.Popen(
            cmd, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT, 
            text=True, 
            bufsize=1,
            universal_newlines=True
        )
        
        # Stream output in real-time
        while True:
            line = process.stdout.readline()
            if not line:
                break
            print(line, end='')
            sys.stdout.flush()
        
        # Wait for process to complete
        process.wait()
        
        print("-" * 50)
        print(f"[INFO] Training process exited with code: {process.returncode}")
        
        if process.returncode == 0:
            print("[SUCCESS] Training completed successfully!")
        else:
            print("[ERROR] Training failed. Check the logs above for details.")
        
        return process.returncode == 0
        
    except KeyboardInterrupt:
        print("\n[INFO] Training interrupted by user")
        if 'process' in locals():
            process.terminate()
        return False
    except Exception as e:
        print(f"[ERROR] Unexpected error during training: {e}")
        return False

def main():
    """
    Main function to run the training with all checks.
    """
    print("=== YOLOX Training Script ===")
    print(f"YOLOX Path: {YOLOX_PATH}")
    print(f"Config: {CONFIG_NAME}")
    print()
    
    # Check if YOLOX is installed
    if not os.path.exists(YOLOX_PATH):
        print(f"[ERROR] YOLOX not found at: {YOLOX_PATH}")
        return
    
    # Start training
    success = train_yolox(CONFIG_NAME)
    
    if success:
        print("\n🎉 Training completed successfully!")
        print("Check the YOLOX_outputs directory for results.")
    else:
        print("\n❌ Training failed or was interrupted.")
        print("Please check the error messages above and fix any issues.")

if __name__ == "__main__":
    main()

In [26]:
#!/usr/bin/env python3
"""
Minimal YOLOX Training Script with Focused Fix
"""

import os
import subprocess
import sys

YOLOX_PATH = r"C:\Users\aarnaizl\Documents\YOLOX"
CONFIG_NAME = "yolox_m"

def train_yolox_minimal():
    """
    Train YOLOX with focused fixes applied.
    """
    exp_file = os.path.join(YOLOX_PATH, "exps", "default", f"{CONFIG_NAME}.py")
    
    if not os.path.exists(exp_file):
        print(f"[ERROR] Config file not found: {exp_file}")
        return False
    
    cmd = [
        "python", os.path.join(YOLOX_PATH, "tools", "train.py"),
        "-f", exp_file,
        "-d", "1",           # Use 1 GPU
        "-b", "8",           # Smaller batch size for stability
        "--fp16"             # Use mixed precision
    ]
    
    print("[INFO] Training YOLOX with focused fix:")
    print(" ".join(cmd))
    print("-" * 50)
    
    try:
        process = subprocess.Popen(
            cmd, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT, 
            text=True, 
            bufsize=1,
            universal_newlines=True
        )
        
        # Stream output in real-time
        error_count = 0
        while True:
            line = process.stdout.readline()
            if not line:
                break
            print(line, end='')
            sys.stdout.flush()
            
            # Count evaluation errors but don't stop training
            if "Error" in line and "evaluation" in line.lower():
                error_count += 1
                if error_count <= 3:  # Only warn for first few errors
                    print(f"[WARNING] Evaluation error #{error_count} detected but training continues...")
        
        process.wait()
        
        print("-" * 50)
        print(f"[INFO] Training process exited with code: {process.returncode}")
        
        if process.returncode == 0:
            print("[SUCCESS] Training completed!")
        else:
            print("[INFO] Training finished - check for saved checkpoints")
        
        # Check if any checkpoints were saved
        output_dir = os.path.join(YOLOX_PATH, "YOLOX_outputs")
        if os.path.exists(output_dir):
            print(f"[INFO] Check {output_dir} for saved model files")
        
        return True
        
    except KeyboardInterrupt:
        print("\n[INFO] Training interrupted by user")
        if 'process' in locals():
            process.terminate()
        return False
    except Exception as e:
        print(f"[ERROR] Unexpected error: {e}")
        return False

if __name__ == "__main__":
    print("=== YOLOX Minimal Training with Focused Fix ===")
    train_yolox_minimal()

=== YOLOX Minimal Training with Focused Fix ===
[INFO] Training YOLOX with focused fix:
python C:\Users\aarnaizl\Documents\YOLOX\tools\train.py -f C:\Users\aarnaizl\Documents\YOLOX\exps\default\yolox_m.py -d 1 -b 8 --fp16
--------------------------------------------------
c:\users\aarnaizl\documents\yolox\yolox\core\trainer.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=args.fp16)
2025-06-10 16:52:02 | INFO     | yolox.core.trainer:132 - args: Namespace(experiment_name='yolo_signal_test', name=None, dist_backend='nccl', dist_url=None, batch_size=8, devices=1, exp_file='C:\\Users\\aarnaizl\\Documents\\YOLOX\\exps\\default\\yolox_m.py', resume=False, ckpt=None, start_epoch=None, num_machines=1, machine_rank=0, fp16=True, cache=None, occupy=False, logger='tensorboard', opts=[])
2025-06-10 16:52:02 | INFO     | yolox.core.trainer:133 - exp value:
╒═══════

In [24]:
#!/usr/bin/env python3
"""
Clean YOLOX Fix - Restore original and apply minimal patch
"""

import os
import shutil

def restore_original_file():
    """
    Restore the original coco_evaluator.py file from backup.
    """
    yolox_path = r"C:\Users\aarnaizl\Documents\YOLOX"
    evaluator_file = os.path.join(yolox_path, "yolox", "evaluators", "coco_evaluator.py")
    
    # Look for backup files
    backup_files = [
        evaluator_file + ".backup_manual",
        evaluator_file + ".backup_focused", 
        evaluator_file + ".backup",
        evaluator_file + ".original_backup"
    ]
    
    for backup_file in backup_files:
        if os.path.exists(backup_file):
            print(f"[INFO] Restoring from backup: {backup_file}")
            shutil.copy2(backup_file, evaluator_file)
            print("[SUCCESS] Original file restored!")
            return True
    
    print("[ERROR] No backup file found")
    print("[INFO] You may need to reinstall YOLOX or restore manually")
    return False

def apply_minimal_fix():
    """
    Apply only the minimal fix needed for the tuple/float division error.
    """
    yolox_path = r"C:\Users\aarnaizl\Documents\YOLOX"
    evaluator_file = os.path.join(yolox_path, "yolox", "evaluators", "coco_evaluator.py")
    
    if not os.path.exists(evaluator_file):
        print(f"[ERROR] Evaluator file not found: {evaluator_file}")
        return False
    
    # Read the file
    with open(evaluator_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # Find the problematic line and fix it
    fixed = False
    for i, line in enumerate(lines):
        if "self.img_size / float(img_h), self.img_size / float(img_w)" in line:
            # Replace this line with a fixed version
            indent = line[:len(line) - len(line.lstrip())]  # Get the indentation
            new_line = f"{indent}# MINIMAL FIX: Handle tuple img_size\n"
            new_line += f"{indent}img_size_val = self.img_size[0] if isinstance(self.img_size, (list, tuple)) else self.img_size\n"
            new_line += f"{indent}scale = min(img_size_val / float(img_h), img_size_val / float(img_w))\n"
            
            lines[i] = new_line
            fixed = True
            break
    
    if not fixed:
        print("[ERROR] Could not find the problematic line to fix")
        return False
    
    # Write the fixed file
    try:
        with open(evaluator_file, 'w', encoding='utf-8') as f:
            f.writelines(lines)
        
        print("[SUCCESS] Minimal fix applied!")
        return True
        
    except Exception as e:
        print(f"[ERROR] Failed to write fixed file: {e}")
        return False

def create_simple_training_script():
    """
    Create a simple training script with the original parameters.
    """
    
    script_content = '''#!/usr/bin/env python3
"""
Simple YOLOX Training Script
"""

import os
import subprocess
import sys

YOLOX_PATH = r"C:\\Users\\aarnaizl\\Documents\\YOLOX"
CONFIG_NAME = "yolox_m"

def train_yolox_simple():
    """
    Train YOLOX with minimal parameters.
    """
    exp_file = os.path.join(YOLOX_PATH, "exps", "default", f"{CONFIG_NAME}.py")
    
    if not os.path.exists(exp_file):
        print(f"[ERROR] Config file not found: {exp_file}")
        return False
    
    # Use your original command but with smaller batch size
    cmd = [
        "python", os.path.join(YOLOX_PATH, "tools", "train.py"),
        "-f", exp_file,
        "-d", "1",
        "-b", "4",  # Reduced batch size
        "--fp16",
        "-o",
        "--cache"
    ]
    
    print("[INFO] Starting YOLOX training:")
    print(" ".join(cmd))
    print("-" * 50)
    
    try:
        # Simple subprocess call
        result = subprocess.run(cmd, capture_output=False, text=True)
        
        print("-" * 50)
        print(f"[INFO] Training exited with code: {result.returncode}")
        
        if result.returncode == 0:
            print("[SUCCESS] Training completed!")
        
        return result.returncode == 0
        
    except KeyboardInterrupt:
        print("\\n[INFO] Training interrupted by user")
        return False
    except Exception as e:
        print(f"[ERROR] Training error: {e}")
        return False

if __name__ == "__main__":
    print("=== Simple YOLOX Training ===")
    train_yolox_simple()
'''
    
    script_file = "train_simple.py"
    with open(script_file, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"[SUCCESS] Created simple training script: {script_file}")
    return script_file

def main():
    """
    Clean fix process: restore original, apply minimal fix, create simple script.
    """
    print("=== Clean YOLOX Fix Process ===")
    print("This will:")
    print("1. Restore the original evaluator file")
    print("2. Apply only the minimal fix needed")
    print("3. Create a simple training script")
    print()
    
    # Step 1: Restore original
    print("Step 1: Restoring original file...")
    if not restore_original_file():
        print("[ERROR] Could not restore original file")
        print("You may need to reinstall YOLOX or manually restore the file")
        return False
    
    # Step 2: Apply minimal fix
    print("\\nStep 2: Applying minimal fix...")
    if not apply_minimal_fix():
        print("[ERROR] Could not apply minimal fix")
        return False
    
    # Step 3: Create simple training script
    print("\\nStep 3: Creating simple training script...")
    script_file = create_simple_training_script()
    
    print("\\n" + "="*50)
    print("🎉 Clean fix process completed!")
    print("\\nNext steps:")
    print(f"1. Run: python {script_file}")
    print("2. This should work without indentation errors")
    print("3. The fix only changes the problematic line")
    print("\\nIf this still doesn't work, you may need to:")
    print("- Reinstall YOLOX from scratch")
    print("- Check your Python environment")
    
    return True

if __name__ == "__main__":
    main()

=== Clean YOLOX Fix Process ===
This will:
1. Restore the original evaluator file
2. Apply only the minimal fix needed
3. Create a simple training script

Step 1: Restoring original file...
[INFO] Restoring from backup: C:\Users\aarnaizl\Documents\YOLOX\yolox\evaluators\coco_evaluator.py.backup_manual
[SUCCESS] Original file restored!
\nStep 2: Applying minimal fix...
[ERROR] Could not find the problematic line to fix
[ERROR] Could not apply minimal fix
